## **Import Libraries**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## **Web Scraping**

### **Olympia 21**

In [3]:
!pip3 install beautifulsoup4
!pip3 install requests

In [4]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata

In [6]:
#Request content from page
url = "https://vi.wikipedia.org/wiki/%C4%90%C6%B0%E1%BB%9Dng_l%C3%AAn_%C4%91%E1%BB%89nh_Olympia_n%C4%83m_th%E1%BB%A9_21"
response = requests.get(url)
soup = BeautifulSoup(response.content)
soup.title

<title>Đường lên đỉnh Olympia năm thứ 21 – Wikipedia tiếng Việt</title>

In [12]:
#Extract all the tables
html_tables = soup.find_all(name="table")
first_table = 3
last_table = 54

In [81]:
#Initalize dictionary for data collection
columns = ["Tên", "Trường", "Tỉnh thành", "Khởi động", "VCNV", "Tăng tốc", "Về đích", "Tổng điểm"]
df_dict_2021 = dict.fromkeys(columns)
for col in columns:
  df_dict_2021[col] = []
print(df_dict_2021)

{'Tên': [], 'Trường': [], 'Tỉnh thành': [], 'Khởi động': [], 'VCNV': [], 'Tăng tốc': [], 'Về đích': [], 'Tổng điểm': []}


In [82]:
for table in html_tables[first_table: last_table + 1]:
  mark_title_row = len(table.find_all("tr")[0]) == 2
  for (index, row) in enumerate(table.find_all("tr")):
    if (mark_title_row and index > 1) or (not mark_title_row and index > 0):
      datas = row.find_all('td')
      
      #Get candidate name
      name = datas[0].text.strip()
      name = re.sub(r'\[.*\]', '', name)
      df_dict_2021['Tên'].append(name)

      a_tags = datas[1].find_all('a')
      #Get school
      school = a_tags[0].text.strip()
      school = re.sub(r'\[.*\]', '', school)
      df_dict_2021['Trường'].append(school)

      #Get province
      if len(a_tags) == 1:
        df_dict_2021['Tỉnh thành'].append(None)
      else:
        province = a_tags[1].text.strip()
        province = re.sub(r'\[.*\]', '', province)
        df_dict_2021['Tỉnh thành'].append(province)

      #Get points
      labels = ['Khởi động', 'VCNV', 'Tăng tốc', 'Về đích', 'Tổng điểm']
      for i in range(2, 7):
        point = datas[i].text.strip()
        point = re.sub(r'\[.*\]', '', point)
        df_dict_2021[labels[i - 2]].append(point)

In [88]:
df_2021 = pd.DataFrame(df_dict_2021)

#Handle missing province value
index_list = list(df_2021[df_2021["Tỉnh thành"].isnull()].index)
df_2021.iloc[119]['Trường'] = 'THPT Chuyên Lê Quý Đôn'
df_2021.iloc[152]['Trường'] = 'THPT Chuyên Lê Quý Đôn'
missing_provinces = ['TP. Hồ Chí Minh'] * 2 + ['Hà Nội'] * 3 + ['TP. Hồ Chí Minh'] + ['Bình Định'] + ['Hà Nội'] * 2 + ['Bình Định'] + ['Hà Nội'] * 4
for (index, value) in enumerate(index_list):
  df_2021.iloc[value]['Tỉnh thành'] = missing_provinces[index]

df_2021

,Tên,Trường,Tỉnh thành,Khởi động,VCNV,Tăng tốc,Về đích,Tổng điểm
0,Võ Phương Nam,THPT Chuyên Bảo Lộc,Lâm Đồng,80,30,120,25,255
1,Nguyễn Hoàng Khánh,THPT Bạch Đằng,Quảng Ninh,110,20,100,90,320
2,Đoàn Đặng Phương Nam,Phổ thông Năng khiếu,TP. Hồ Chí Minh,60,20,20,-60,40
3,Nguyễn Quang Huy,THPT Ngô Quyền - Ba Vì,Hà Nội,100,10,110,100,320
4,Lê Nguyên Hạo,THPT An Ninh,Long An,70,20,50,20,160
...,...,...,...,...,...,...,...,...
203,Bùi Đức Đăng,THPT Chuyên Hạ Long,Quảng Ninh,50,10,130,40,230
204,Bùi Đức Đăng,THPT Chuyên Hạ Long,Quảng Ninh,60,10,80,80,230
205,Nguyễn Đình Duy Anh,THPT Chuyên Phan Bội Châu,Nghệ An,50,70,60,70,250
206,Trương Công Minh,THPT Chuyên Lê Khiết,Quảng Ngãi,60,0,120,10,190


In [104]:
#Add match index to dataframe
week_indexes = (([1] * 4 + [2] * 4 + [3] * 4 + ['X'] * 4) * 3 + ['X'] * 4) * 4
month_indexes = ([1] * 16 + [2] * 16 + [3] * 16 + ['X'] * 4) * 4
quarter_indexes = [1] * 52 + [2] * 52 + [3] * 52 + [4] * 52

df_2021['Tuần'] = week_indexes
df_2021['Tháng'] = month_indexes
df_2021['Quý'] = quarter_indexes
df_2021

,Tên,Trường,Tỉnh thành,Khởi động,VCNV,Tăng tốc,Về đích,Tổng điểm,Tuần,Tháng,Quý
0,Võ Phương Nam,THPT Chuyên Bảo Lộc,Lâm Đồng,80,30,120,25,255,1,1,1
1,Nguyễn Hoàng Khánh,THPT Bạch Đằng,Quảng Ninh,110,20,100,90,320,1,1,1
2,Đoàn Đặng Phương Nam,Phổ thông Năng khiếu,TP. Hồ Chí Minh,60,20,20,-60,40,1,1,1
3,Nguyễn Quang Huy,THPT Ngô Quyền - Ba Vì,Hà Nội,100,10,110,100,320,1,1,1
4,Lê Nguyên Hạo,THPT An Ninh,Long An,70,20,50,20,160,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...
203,Bùi Đức Đăng,THPT Chuyên Hạ Long,Quảng Ninh,50,10,130,40,230,X,3,4
204,Bùi Đức Đăng,THPT Chuyên Hạ Long,Quảng Ninh,60,10,80,80,230,X,X,4
205,Nguyễn Đình Duy Anh,THPT Chuyên Phan Bội Châu,Nghệ An,50,70,60,70,250,X,X,4
206,Trương Công Minh,THPT Chuyên Lê Khiết,Quảng Ngãi,60,0,120,10,190,X,X,4


### **Olympia 20**

In [108]:
#Request content from page
url = "https://vi.wikipedia.org/wiki/%C4%90%C6%B0%E1%BB%9Dng_l%C3%AAn_%C4%91%E1%BB%89nh_Olympia_n%C4%83m_th%E1%BB%A9_20"
response = requests.get(url)
soup = BeautifulSoup(response.content)
soup.title

<title>Đường lên đỉnh Olympia năm thứ 20 – Wikipedia tiếng Việt</title>

In [111]:
#Extract all the tables
html_tables = soup.find_all(name="table")

In [127]:
#Initalize dictionary for data collection
columns = ["Tên", "Trường", "Tỉnh thành", "Khởi động", "VCNV", "Tăng tốc", "Về đích", "Tổng điểm"]
df_dict_2020 = dict.fromkeys(columns)
for col in columns:
  df_dict_2020[col] = []
print(df_dict_2020)

{'Tên': [], 'Trường': [], 'Tỉnh thành': [], 'Khởi động': [], 'VCNV': [], 'Tăng tốc': [], 'Về đích': [], 'Tổng điểm': []}


In [128]:
for table in html_tables[first_table: last_table + 1]:
  for (index, row) in enumerate(table.find_all("tr")):
    if index > 0:
      datas = row.find_all('td')
      
      #Get candidate name
      name = datas[0].text.strip()
      name = re.sub(r'\[.*\]', '', name)
      df_dict_2020['Tên'].append(name)

      a_tags = datas[1].find_all('a')
      #Get school
      school = a_tags[0].text.strip()
      school = re.sub(r'\[.*\]', '', school)
      df_dict_2020['Trường'].append(school)

      #Get province
      if len(a_tags) == 1:
        df_dict_2020['Tỉnh thành'].append(None)
      else:
        province = a_tags[1].text.strip()
        province = re.sub(r'\[.*\]', '', province)
        df_dict_2020['Tỉnh thành'].append(province)

      #Get points
      labels = ['Khởi động', 'VCNV', 'Tăng tốc', 'Về đích', 'Tổng điểm']
      for i in range(2, 7):
        point = datas[i].text.strip()
        point = re.sub(r'\[.*\]', '', point)
        df_dict_2020[labels[i - 2]].append(point)

In [131]:
df_2020 = pd.DataFrame(df_dict_2020)
df_2020['Tuần'] = week_indexes
df_2020['Tháng'] = month_indexes
df_2020['Quý'] = quarter_indexes
df_2020

,Tên,Trường,Tỉnh thành,Khởi động,VCNV,Tăng tốc,Về đích,Tổng điểm,Tuần,Tháng,Quý
0,Nguyễn Huy,THPT Chuyên Huỳnh Mẫn Đạt,Kiên Giang,50,10,140,10,210,1,1,1
1,Trần Phạm Anh Tuấn,THPT Lương Thế Vinh,Quảng Bình,50,90,90,5,235,1,1,1
2,Nguyễn Xuân Huy,THPT Thăng Long,Hà Nội,70,10,80,90,250,1,1,1
3,Nguyễn Tuấn Anh,THPT Triệu Quang Phục,Hưng Yên,80,10,70,-60,100,1,1,1
4,Hoàng Anh Quân,THPT Nguyễn Huệ,Thừa Thiên - Huế,90,10,150,110,360,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...
203,Bùi Nữ Minh Ngọc,THPT Chuyên Nguyễn Tất Thành,Kon Tum,20,0,20,30,70,X,3,4
204,Tạ Quang Hưng,THPT Chuyên Bắc Ninh,Bắc Ninh,80,20,40,70,210,X,X,4
205,Lưu Đào Dũng Trí,THPT Chuyên Đại học Sư phạm Hà Nội,Hà Nội,70,60,90,60,280,X,X,4
206,Lê Minh,THPT Sơn Tây,Hà Nội,50,10,80,-40,100,X,X,4


### **Olympia 19**

In [132]:
#Request content from page
url = "https://vi.wikipedia.org/wiki/%C4%90%C6%B0%E1%BB%9Dng_l%C3%AAn_%C4%91%E1%BB%89nh_Olympia_n%C4%83m_th%E1%BB%A9_19"
response = requests.get(url)
soup = BeautifulSoup(response.content)
soup.title

<title>Đường lên đỉnh Olympia năm thứ 19 – Wikipedia tiếng Việt</title>

In [135]:
#Extract all the tables
html_tables = soup.find_all(name="table")

In [136]:
#Initalize dictionary for data collection
columns = ["Tên", "Trường", "Tỉnh thành", "Khởi động", "VCNV", "Tăng tốc", "Về đích", "Tổng điểm"]
df_dict_2019 = dict.fromkeys(columns)
for col in columns:
  df_dict_2019[col] = []
print(df_dict_2019)

{'Tên': [], 'Trường': [], 'Tỉnh thành': [], 'Khởi động': [], 'VCNV': [], 'Tăng tốc': [], 'Về đích': [], 'Tổng điểm': []}


In [137]:
for table in html_tables[first_table: last_table + 1]:
  for (index, row) in enumerate(table.find_all("tr")):
    if index > 0:
      datas = row.find_all('td')
      
      #Get candidate name
      name = datas[0].text.strip()
      name = re.sub(r'\[.*\]', '', name)
      df_dict_2019['Tên'].append(name)

      a_tags = datas[1].find_all('a')
      #Get school
      school = a_tags[0].text.strip()
      school = re.sub(r'\[.*\]', '', school)
      df_dict_2019['Trường'].append(school)

      #Get province
      if len(a_tags) == 1:
        df_dict_2019['Tỉnh thành'].append(None)
      else:
        province = a_tags[1].text.strip()
        province = re.sub(r'\[.*\]', '', province)
        df_dict_2019['Tỉnh thành'].append(province)

      #Get points
      labels = ['Khởi động', 'VCNV', 'Tăng tốc', 'Về đích', 'Tổng điểm']
      for i in range(2, 7):
        point = datas[i].text.strip()
        point = re.sub(r'\[.*\]', '', point)
        df_dict_2019[labels[i - 2]].append(point)

In [140]:
df_2019 = pd.DataFrame(df_dict_2019)
df_2019['Tuần'] = week_indexes
df_2019['Tháng'] = month_indexes
df_2019['Quý'] = quarter_indexes
df_2019

,Tên,Trường,Tỉnh thành,Khởi động,VCNV,Tăng tốc,Về đích,Tổng điểm,Tuần,Tháng,Quý
0,Cấn Thái Hoàng,THPT Thạch Thất,Hà Nội,60,50,160,20,290,1,1,1
1,Vũ Thành Vinh,THPT Hữu Lũng,Lạng Sơn,50,0,80,0,130,1,1,1
2,Lê Đức Thành,THPT Nguyễn Thị Minh Khai,Hà Tĩnh,80,0,70,-20,130,1,1,1
3,Đinh Đoàn Xuân Phương,THPT Chuyên Võ Nguyên Giáp,Quảng Bình,80,0,20,30,130,1,1,1
4,Bùi Hậu Giang,THPT Chuyên Lê Quý Đôn,Đà Nẵng,60,90,120,60,330,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...
203,Trần Lệ Mỹ,THPT Yên Lập,Phú Thọ,30,10,40,-70,10,X,3,4
204,Nguyễn Đức Hiếu,THPT Chuyên Thái Bình,Thái Bình,40,10,50,-55,45,X,X,4
205,Đoàn Nam Thắng,THPT Chuyên Nguyễn Du,Đắk Lắk,40,70,70,30,210,X,X,4
206,Vũ Công Minh,THPT Quang Trung,Hải Dương,40,10,90,20,160,X,X,4


### **Export as .csv**

In [141]:
df_2021.to_csv('./o21.csv')
df_2020.to_csv('./o20.csv')
df_2019.to_csv('./o19.csv')